In [10]:
# NOTE: export SPARK_DRIVER_MEMORY=1g

In [11]:
import keras
from keras.applications import ResNet50
from keras.applications import imagenet_utils

* For resnet, we only support NCHW and Keras 1.2.2 for now

In [12]:
keras.backend.set_image_dim_ordering("th")

* Create ResNet50 using Keras

In [13]:
keras_model = ResNet50(weights="imagenet")

/home/lizhichao/anaconda3/envs/keras2/lib/python3.5/site-packages/keras/applications/resnet50.py:231: UserWarning: You are using the TensorFlow backend, yet you are using the Theano image dimension ordering convention (`image_dim_ordering="th"`). For best performance, set `image_dim_ordering="tf"` in your Keras config at ~/.keras/keras.json.
  warnings.warn('You are using the TensorFlow backend, yet you '


* Init BigDL and spark environment

In [5]:
from bigdl.keras.backend import *
from bigdl.transform.vision.image import *

redire_spark_logs()
show_bigdl_info_logs()
init_engine()
sparkConf = create_spark_conf().setAppName("test model")
sc = get_spark_context(sparkConf)

Adding /home/lizhichao/anaconda3/envs/keras2/lib/python3.5/site-packages/bigdl/share/lib/bigdl-0.6.0-jar-with-dependencies.jar to BIGDL_JARS
Prepending /home/lizhichao/anaconda3/envs/keras2/lib/python3.5/site-packages/bigdl/share/conf/spark-bigdl.conf to sys.path


* Load a Keras definition and weights to BigDL.

In [6]:
bmodel = DefinitionLoader.from_kmodel(keras_model)
WeightLoader.load_weights_from_kmodel(bmodel, keras_model)

creating: createInput
creating: createSequential
creating: createPadding
creating: createPadding
creating: createPadding
creating: createPadding
creating: createSpatialConvolution


/home/lizhichao/anaconda3/envs/keras2/lib/python3.5/site-packages/bigdl/keras/converter.py:791: UserWarning: Cannot find dim_ordering from json definition. Using the default instead.
  warnings.warn("Cannot find dim_ordering from json definition. Using the default instead.")


creating: createXavier
creating: createZeros
creating: createSpatialBatchNormalization
creating: createReLU
creating: createSpatialMaxPooling
creating: createSpatialConvolution
creating: createXavier
creating: createZeros
creating: createSpatialBatchNormalization
creating: createReLU
creating: createSpatialConvolution
creating: createXavier
creating: createZeros
creating: createSpatialBatchNormalization
creating: createReLU
creating: createSpatialConvolution
creating: createXavier
creating: createZeros
creating: createSpatialConvolution
creating: createXavier
creating: createZeros
creating: createSpatialBatchNormalization
creating: createSpatialBatchNormalization
creating: createCAddTable
creating: createReLU
creating: createSpatialConvolution
creating: createXavier
creating: createZeros
creating: createSpatialBatchNormalization
creating: createReLU
creating: createSpatialConvolution
creating: createXavier
creating: createZeros
creating: createSpatialBatchNormalization
creating: create

* Load data into ImageFrame

In [7]:
image_frame = ImageFrame.read("image-net", sc=sc)

* Define prepocess transformer

In [14]:
transformer = Pipeline([BytesToMat(), Resize(256, 256),
                        CenterCrop(224, 224),
                        ChannelNormalize(123.68, 103.939, 116.779),
                        MatToTensor(), ImageFrameToSample()])

creating: createBytesToMat
creating: createResize
creating: createCenterCrop
creating: createChannelNormalize
creating: createMatToTensor
creating: createImageFrameToSample
creating: createPipeline


* Run prediction

In [15]:
preds_frame = bmodel.predict_image(image_frame.transform(transformer))

Py4JJavaError: An error occurred while calling o27.modelPredictImage.
: java.lang.OutOfMemoryError: Java heap space
	at java.util.Arrays.copyOf(Arrays.java:3236)
	at java.io.ByteArrayOutputStream.grow(ByteArrayOutputStream.java:118)
	at java.io.ByteArrayOutputStream.ensureCapacity(ByteArrayOutputStream.java:93)
	at java.io.ByteArrayOutputStream.write(ByteArrayOutputStream.java:153)
	at org.apache.spark.util.ByteBufferOutputStream.write(ByteBufferOutputStream.scala:41)
	at java.io.ObjectOutputStream$BlockDataOutputStream.drain(ObjectOutputStream.java:1877)
	at java.io.ObjectOutputStream$BlockDataOutputStream.setBlockDataMode(ObjectOutputStream.java:1786)
	at java.io.ObjectOutputStream.writeObject0(ObjectOutputStream.java:1189)
	at java.io.ObjectOutputStream.writeObject(ObjectOutputStream.java:348)
	at org.apache.spark.serializer.JavaSerializationStream.writeObject(JavaSerializer.scala:43)
	at org.apache.spark.serializer.JavaSerializerInstance.serialize(JavaSerializer.scala:100)
	at org.

* Show result of the first image

In [10]:
preds = preds_frame.get_predict().take(1)

for pre in preds:
    path, pre_score = pre
    P = imagenet_utils.decode_predictions(np.expand_dims(pre_score, axis=0))
    print("Prediction for {}".format(path))
    # Display the top-5 prediction
    for (i, (imagenetID, label, prob)) in enumerate(P[0]):
        print("    {}. {}: {:.2f}%".format(i + 1, label, prob * 100))

Prediction for hdfs://ng3afac14-master-instance-0oof254w.novalocal:8020/user/root/image-net/158516268_796c3e047d.jpg
    1. sorrel: 96.77%
    2. ox: 0.72%
    3. llama: 0.52%
    4. gazelle: 0.42%
    5. basenji: 0.23%


* While the origin image is
![scalars](image-net/158516268_796c3e047d.jpg)